In [1]:
import os

In [2]:
%pwd

'd:\\food_not_food\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\food_not_food'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    dataset_name: str
    local_data_file: Path
    unzip_dir: Path


In [6]:
from classifier.constants import *
from classifier.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        return DataIngestionConfig(
            root_dir=config.root_dir,
            dataset_name=config.dataset_name,  
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )



In [10]:
import os
import zipfile
from pathlib import Path
from dotenv import load_dotenv
from classifier import logger
from classifier.utils.common import get_size
from classifier.entity.config_entity import DataIngestionConfig

load_dotenv()
from kaggle.api.kaggle_api_extended import KaggleApi

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            api = KaggleApi()
            api.authenticate()

            logger.info(f"Downloading dataset: {self.config.dataset_name}")
            api.dataset_download_files(
                dataset=self.config.dataset_name,
                path=self.config.root_dir,
                # filename=os.path.basename(self.config.local_data_file),
                unzip=False
            )
            downloaded_zip_path = os.path.join(self.config.root_dir, f"{self.config.dataset_name.split('/')[-1]}.zip")
            os.rename(downloaded_zip_path, self.config.local_data_file)
            logger.info(f"Downloaded and renamed to: {self.config.local_data_file}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        os.makedirs(self.config.unzip_dir, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(self.config.unzip_dir)
            logger.info(f"Extracted to {self.config.unzip_dir}")


In [11]:

try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-05-20 22:08:49,291: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-05-20 22:08:49,294: INFO: common: yaml file: params.yaml loaded successfully]
[2025-05-20 22:08:49,296: INFO: common: created directory at: artifacts]
[2025-05-20 22:08:49,298: INFO: common: created directory at: artifacts/data_ingestion]
[2025-05-20 22:08:49,300: INFO: 3941196050: File already exists of size: ~ 285337 KB]
[2025-05-20 22:08:58,082: INFO: 3941196050: Extracted to artifacts/data_ingestion]
